## Implementación de una simulación DES

En este _notebook_ nos enfocaremos en revisar una **posible** manera de implementar una simulación basada en eventos discretos.

### Eventos

Partamos por definir (programáticamente) lo que es un evento. Consideremos que:

- Un evento tiene un tiempo de ocurrencia
- Queremos que ocurra algo en ese momento
- Nos gustaría poder identificar un evento por un identificador único

Con eso, podemos construir un a clase `Evento` que tenga cada uno de esos atributos:

- `id`: nos ayudará a identificar un evento en forma única
- `tiempo`: indicará el momento en que el evento debe ejecutarse
- `función`: contendrá lo que queramos que se ejecute en ese evento

Además, nos gustaría poder ordenar los eventos según su `tiempo`, y poder comparar el evento con un `int` o `float` para saber si un evento ocurre antes de cierto instante. Para ello, implementaremos [`__lt__`](https://docs.python.org/3/reference/datamodel.html#object.__lt__) (_less than_).

In [1]:
from itertools import count


class Evento:
    ids = count(start=0)

    def __init__(self, tiempo, función):
        self.id = next(self.ids)
        self.tiempo = tiempo
        self.función = función

    def __repr__(self):
        template = "Evento id={}, tiempo={}"
        return template.format(self.id, self.tiempo)

    def __lt__(self, other):
        if isinstance(other, Evento):
            return self.tiempo < other.tiempo
        return self.tiempo < other

Opcionalmente, se le puede colocar una descripción al evento, de tal manera de poder imprimir en pantalla algo más completo.

### Flujo general de una simulación

Ahora que tenemos eventos, intentemos construir el motor principal de una simulación con una clase `MotorSimulación`. Recordemos que una simulación debe tener:

- Un tiempo de inicio
- Un tiempo en que la simulación debe finalizar
- Una lista de eventos a ejecutar, ordenada por el tiempo de ocurrencia

In [2]:
import math


class MotorSimulación:
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self.eventos = []

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)

Tenemos todos nuestros atributos básicos, pero nos falta un detalle: la lista de eventos debe mantenerse ordenada. No es materia de este curso saber cómo hacer esto de la forma más eficiente, por lo que sólo nos aseguraremos que, cada vez que se acceda a esta lista, se ordene por el tiempo de los eventos. Haremos esto vía _properties_.

También aprovecharemos de poner los otros atributos como _properties_, ya que no querremos modificarlos desde fuera de la clase.

In [3]:
class MotorSimulación:
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self._tiempo_simulación = tiempo_inicio
        self._tiempo_fin = tiempo_fin
        self._eventos = []

    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort()
        return self._eventos

    @property
    def tiempo_fin(self):
        return self._tiempo_fin

    @property
    def tiempo_simulación(self):
        return self._tiempo_simulación

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)

Ahora, nos falta el flujo principal. Recordemos que el algoritmo general de una simulación DES es el siguiente:

    MIENTRAS la lista de eventos no esté vacía y el tiempo de simulación no termine:
        tomar un evento desde el principio de la lista de eventos
        avanzar el tiempo de simulación al tiempo del evento
        simular el evento
        
Implementaremos este algoritmo en una función llamada `run` de nuestra clase `MotorSimulación`:

In [4]:
class MotorSimulación:
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self._tiempo_simulación = tiempo_inicio
        self._tiempo_fin = tiempo_fin
        self._eventos = []

    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort()
        return self._eventos

    @property
    def tiempo_fin(self):
        return self._tiempo_fin

    @property
    def tiempo_simulación(self):
        return self._tiempo_simulación

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)

    def run(self):
        """Ejecuta la simulación."""
        # Revisamos que haya un evento
        # Y que este evento esté dentro del límite de tiempo
        while self.eventos and self.eventos[0] < self.tiempo_fin:
            # Sacamos el primer evento de la lista
            evento = self.eventos.pop(0)
            # Avanzamos el tiempo de simulación al tiempo del evento
            self._tiempo_simulación = evento.tiempo
            print("-" * 80)
            print("TIEMPO: {:.2f}".format(self.tiempo_simulación))
            # Simulamos el evento
            evento.función()

### Objetos de la simulación

Los objetos de nuestra simulación, deberían cumplir:
- Recibir una instancia del motor de la simulación, para que pueda programar eventos y revisar el instante actual.
- Que se realicen ajustes iniciales. Esto puede incluir cualquier configuración, pero también agregar eventos iniciales. Es bueno recordar que la simulación debe comenzar con eventos **ya programados**.

Abajo podemos ver cómo se vería – en general – una clase de nuestro modelo de simulación.

In [5]:
class ObjetoSimulación:
    def __init__(self, motor_simulación, *args, **kwargs):
        self.motor_simulación = motor_simulación
        # Hacer lo que tengamos que hacer
        self.configurar()

    def configurar(self):
        """Método de configuración inicial"""
        # Agregar eventos iniciales a la simulación (si es que aplica)
        evento = Evento(15, lambda: print("Evento de saludar al mundo!"))
        self.motor_simulación.agregar_evento(evento)

    def recibir_cosas(self, cosa):
        """
        Recibe una cosa para hacer algo con ella.
        """
        # Acá deberíamos procesar qué hacer con esta cosa que recibimos
        # Esto podría derivar, incluso, en programar un evento.
        pass

    def imprimir_estadísticas(self):
        """Imprime estadísticas finales"""
        print("-" * 80)
        print("Estadísticas")
        print("-" * 80)
        print("En realidad, no hice nada :/")

    def hacer_otra_cosa(self):
        """Esto es otro método del objeto"""
        # Podemos hacer otra cosa acá
        pass

Y creamos nuestro entorno de simulación, donde uniremos todos los componentes necesarios:

In [6]:
class MiSimulación:
    def __init__(self):
        self.motor = MotorSimulación(tiempo_fin=500)
        self.objeto = ObjetoSimulación(self.motor)

    def simular(self):
        self.motor.run()
        self.objeto.imprimir_estadísticas()

In [7]:
simulación = MiSimulación()
simulación.simular()

--------------------------------------------------------------------------------
TIEMPO: 15.00
Evento de saludar al mundo!
--------------------------------------------------------------------------------
Estadísticas
--------------------------------------------------------------------------------
En realidad, no hice nada :/


## Ejemplo de revisión técnica

¡Muy bien! Al parecer tenemos todo lo necesario para volver a programar nuestra revisión técnica, pero ahora con eventos discretos. Partamos con la clase `Taller`.

In [8]:
from collections import deque
from statistics import mean


class Taller:
    """Esta clase modela el taller de revisión técnica."""

    def __init__(self, motor_simulación):
        self.motor_simulación = motor_simulación
        self.cola = deque()
        self.tiempos_atención = []
        self.tiempos_espera = []
        self.vehículo_actual = None

    @property
    def ocupado(self):
        return self.vehículo_actual is not None

    def imprimir_estadísticas(self):
        print("-" * 80)
        print("Estadísticas Taller")
        print("-" * 80)
        print("Vehículos atendidos: {}".format(len(self.tiempos_atención)))
        print("Promedio tiempo de atención: {}".format(
            mean(self.tiempos_atención)))
        print("Promedio tiempo de espera: {}".format(
            mean(self.tiempos_espera)))
        print("Vehículos todavía en cola: {}".format(len(self.cola)))

    def encolar_vehículo(self, vehículo):
        """Acción que ingresa un vehículo a la cola del taller."""
        self.cola.append(vehículo)
        # Imprimimos lo que sucede
        print("Taller:", "{} entró a la cola".format(vehículo))
        # Si no estamos atendiendo a ningún vehículo, atendemos a este
        if not self.ocupado:
            self.atender_siguiente_vehículo()

    def atender_siguiente_vehículo(self):
        """Función para atender al siguiente vehículo de la cola."""
        # Seteamos el vehículo que estamos atendiendo ahora
        self.vehículo_actual = self.cola.popleft()

        # Agregamos el tiempo que el vehículo tuvo que esperar
        # para ser atendido
        self.tiempos_espera.append(self.motor_simulación.tiempo_simulación -
                                   self.vehículo_actual.tiempo_creación)
        # Agregamos el tiempo que el vehículo se demorará en ser revisado
        self.tiempos_atención.append(self.vehículo_actual.tiempo_atención)

        # Calculamos cuando estará listo el vehículo
        tiempo_fin_atención = (self.motor_simulación.tiempo_simulación +
                               self.vehículo_actual.tiempo_atención)
        # Creamos el evento de fin de atención, donde ejecutamos
        # terminar_vehículo
        evento = Evento(tiempo_fin_atención, self.terminar_vehículo)
        # Agregamos el evento a la simulación
        self.motor_simulación.agregar_evento(evento)

        # Imprimimos lo que sucede
        print("Taller:", "{} está siendo atendido".format(
            self.vehículo_actual))

    def terminar_vehículo(self):
        """Función que se ejecuta al terminar de atender un vehículo."""
        print("Taller:", "{} sale".format(self.vehículo_actual))
        # Ya no estamos atendiendo ningún vehículo
        self.vehículo_actual = None
        # Vemos si hay otro vehículo, para comenzar a atenderlo
        if self.cola:
            self.atender_siguiente_vehículo()

Ahora, vamos con el vehículo:

In [9]:
from random import expovariate


class Vehículo:

    ids = count(start=0)

    def __init__(self, motor_simulación, tipo, tasa_media_atención):
        self.id = next(self.ids)
        self.motor_simulación = motor_simulación
        # Colocamos 1 minuto como mínimo de tiempo de atención
        self.tiempo_atención = max(expovariate(tasa_media_atención), 1)
        self.tiempo_creación = self.motor_simulación.tiempo_simulación
        self.tipo = tipo

    def __repr__(self):
        return "{} id={}".format(self.tipo.capitalize(), self.id)

Falta algo, ¿no? Tenemos que hacer "aparecer" los vehículos para que la planta los revise. Podemos hacer esto en el entorno de simulación que creemos, o bien, hacer una clase especializada en esto.

Esta vez, optaremos por hacerlo en una clase separada. Piensa en el caso en que se tenga que hacer aparecer cinco tipos distintos de entidades, tu código podría quedar muy largo, y sin separar bien las responsabilidades de las clases.

In [10]:
from random import choice


class CreadorVehículos:
    def __init__(self, motor_simulación, tasa_media_creación, info_vehículos,
                 taller):
        """
        Inicializa el creador de vehículos.
        
        info_vehículos es un diccionario de str -> float, donde las
        keys son los tipos de vehículos y 
        los valores son las tasas medias de atención de ese tipo.
        """
        self.motor_simulación = motor_simulación
        self.tasa_media_creación = tasa_media_creación
        self.info_vehículos = info_vehículos
        self.taller = taller
        self.configurar()

    def configurar(self):
        # Creamos el evento de la primera creación de vehículo
        self.programar_creación_vehículo()

    def programar_creación_vehículo(self):
        # Vemos en cuánto tiempo (a partir de ahora) liberaremos otro vehículo
        retraso = expovariate(self.tasa_media_creación)
        # Calculamos el tiempo en que se debe liberar el vehículo
        tiempo_evento = self.motor_simulación.tiempo_simulación + retraso
        # Creamos el evento
        evento = Evento(tiempo_evento, self.crear_vehículo)
        # Agregamos el evento a la simulación
        self.motor_simulación.agregar_evento(evento)

    def crear_vehículo(self):
        # Elegimos el tipo de vehículo a crear, junto con su
        # tasa media de atención
        tipo, tasa_media_atención = choice(list(self.info_vehículos.items()))
        # Creamos el vehículo
        vehículo = Vehículo(self.motor_simulación, tipo, tasa_media_atención)
        # Gatillamos la acción de encolar un vehículo en la planta de revisión
        self.taller.encolar_vehículo(vehículo)
        # Programamos la siguiente creación de vehículo
        self.programar_creación_vehículo()

Ahora, ¡a unir todo!

In [11]:
class RevisiónTécnica:
    INFO_VEHÍCULOS = {'moto': 1 / 8, 'auto': 1 / 15, 'camioneta': 1 / 20}
    TASA_VEHÍCULOS = 1 / 5

    def __init__(self):
        self.motor = MotorSimulación(tiempo_fin=50)
        self.taller = Taller(self.motor)
        self.creador_vehículos = CreadorVehículos(
            self.motor, self.TASA_VEHÍCULOS, self.INFO_VEHÍCULOS, self.taller)

    def simular(self):
        self.motor.run()
        self.taller.imprimir_estadísticas()

In [12]:
simulación = RevisiónTécnica()
simulación.simular()

--------------------------------------------------------------------------------
TIEMPO: 8.18
Taller: Auto id=0 entró a la cola
Taller: Auto id=0 está siendo atendido
--------------------------------------------------------------------------------
TIEMPO: 11.59
Taller: Auto id=0 sale
--------------------------------------------------------------------------------
TIEMPO: 13.39
Taller: Auto id=1 entró a la cola
Taller: Auto id=1 está siendo atendido
--------------------------------------------------------------------------------
TIEMPO: 14.46
Taller: Camioneta id=2 entró a la cola
--------------------------------------------------------------------------------
TIEMPO: 18.01
Taller: Moto id=3 entró a la cola
--------------------------------------------------------------------------------
TIEMPO: 25.24
Taller: Auto id=4 entró a la cola
--------------------------------------------------------------------------------
TIEMPO: 45.93
Taller: Auto id=5 entró a la cola
--------------------------

En esta simulación, sacamos un evento y corremos el reloj al tiempo en que ese evento ocurre, por lo que no revisamos los casos en que no ocurre nada. Tenemos eventos de llegadas de vehículos, y de cuando el taller termina de atender a uno. En cada evento se genera un cambio en el estado del sistema, ya sea de la cola de vehículos esperando o del vehículo que está atendiendo el taller.

## Créditos

Material creado por Equipo Docente IIC2233, 2018-1, bajo licencia `Creative Commons Attribution-ShareAlike 4.0 International`